### AML-2203 Advanced Python AI and ML Tools

### Assignment 1: Hypothesis: University towns have their mean housing prices less effected by recessions. Perform the following tasks:

-	Download the data and run a completed set of exploratory data analysis including best possible tasks on that. This may include but not limited to observing the shape, describe the data set, checking the missing values, drawing the profile of the data, checking the distribution type and data types and several other tasks.
-	 Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)

In [1]:
#Import required libraries
import pandas as pd
import re
from scipy.stats import ttest_ind

#Library to import warnings
import warnings
warnings.filterwarnings('ignore')

#### Take the data from wikipedia and copy paste into a txt file and read the text file

In [2]:
with open('university_towns.txt', 'r') as f:

    #Read non-empty lines:
    data = (line.rstrip() for line in f)
    lines = list(line for line in data if line)

    #University states have [edit] in the last, so we can differentiate between states and towns based on this criteria
    #Get the index of states:
    r_idx = [lines.index(line) for line in lines if '[edit]' in line]

    
    #Separating states and university names using wrapping indexes:
    university = []
    region = [lines[i].replace('[edit]', '') for i in r_idx]
    for i in range(len(r_idx)):
        if i != len(r_idx)-1:
            sub = lines[r_idx[i]+1:r_idx[i+1]]
            university.append(sub)
        else:
            sub = lines[r_idx[i]+1:]
            university.append(sub)

    #Create dictionary of university town and related status
    uni = dict(zip(region, university))

In [3]:
data_items = uni.items()
data_list = list(data_items)

df = pd.DataFrame(data_list)
df.columns=['State','University_Town']
df

,State,University_Town
0,Alabama,"[Auburn (Auburn University, Edward Via College..."
1,Alaska,"[Anchorage (University of Alaska Anchorage), F..."
2,Arizona,"[Chandler (Western International University, U..."
3,Gilbert,[Glendale (Thunderbird School of Global Manage...
4,Arkansas,"[Arkadelphia (Henderson State University, Ouac..."
...,...,...
65,Washington,"[Bellevue (Bellevue College, City University o..."
66,West Virginia,"[Athens (Concord University)[8], Beckley (West..."
67,Wisconsin,"[Appleton (Lawrence University), Eau Claire (U..."
68,Milwaukee,"[Oshkosh (University of Wisconsinâ€“Oshkosh), ..."


In [4]:
state_index = {key: [] for key in df['State'].unique()} 
for tmp in range(len(df['State'])):
    for j in df['University_Town'][tmp]:
        state_index[df['State'][tmp]].append(j.split(" ")[0])


In [40]:
optimal_states = {}
for key, value in state_index.items():
    for tmp in value:
        optimal_states[tmp] = [key]

In [6]:
jk = pd.DataFrame.from_dict(optimal_states).T
s = pd.DataFrame({'States': jk[0], 'Town': list(jk.index.values)})
s.reset_index(drop=True, inplace=True)
s.head()

,States,Town
0,Alabama,Auburn
1,Alabama,Birmingham
2,Arkansas,Arkadelphia
3,Alabama,Southside
4,Alabama,Dothan


In [42]:
s.shape

(687, 2)

In [7]:
#Assigning state codes to the states using dictionary Key-Value pairs
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland',
          'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois',
          'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont',
          'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington',
          'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana',
          'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi',
          'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota',
          'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut',
          'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas',
          'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida',
          'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware',
          'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands',
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

#### User defined functions

In [8]:
def house_year_to_quarters():
    
    """This function takes city data as input and convert it into 
    quarters for example; 2001 will be converted 2001Q1, 2001Q2 and 2001Q3
    and so on."""
       
    df = pd.read_csv('city_zhvi_allhomes.csv')
    df3 = pd.read_csv('city_zhvi_allhomes.csv')
    df = df.drop(df.columns[[0] + list(range(6,56))], axis=1)
    df2 = pd.DataFrame(df[['State', 'RegionName']])
    df2.rename(columns={'RegionName':'TownName'},inplace=True)


    df3=df3.drop(df3.columns[[0] + list(range(1,56))], axis=1)

    x=df.columns
    y=list(x[5:])
    new_columns=[]
    for que in y:
        z=re.sub("[\/\[].*?[\/\]]", "", que)
        new_columns.append(z)
    
    df3.columns=new_columns
    housing_df=pd.concat([df2,df3],axis=1).reset_index(drop=True)
    
    #Running a loop from year 2000 to 2020 and assigning quarters based on th months
    for year in range(2000, 2020):
        df2[str(year) + 'Q1'] = housing_df[['1' +  str(year) , '2'  + str(year), '3' +  str(year)]].mean(axis = 1)
        df2[str(year) + 'Q2'] = housing_df[['4' +  str(year) , '5' +  str(year), '6' +  str(year)]].mean(axis = 1)
        df2[str(year) + 'Q3'] = housing_df[['7' +  str(year) , '8' +  str(year), '9' +  str(year)]].mean(axis = 1)
        df2[str(year) + 'Q4'] = housing_df[['10' +  str(year) , '11' +  str(year), '12' +  str(year)]].mean(axis = 1)
        
    year = 2020
    df2[str(year) + 'Q1'] = housing_df[['1' + str(year), '2' + str(year), '3' + str(year)]].mean(axis = 1)

    df2['State'] = [states[state] for state in df2['State']]
    df2 = df2.set_index(['State', 'TownName'])
    new_dataframe = pd.DataFrame(df2)
    
    return new_dataframe

house_year_to_quarters()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1
State,TownName,,,,,,,,,,,,,,,,,,,,,
New York,New York,243464.000000,252251.666667,260138.666667,268484.000000,277120.333333,286576.666667,293365.333333,300368.000000,307615.333333,307932.666667,...,620818.666667,629938.666667,641443.000000,649833.333333,657050.666667,658959.333333,659562.000000,658057.000000,654645.666667,653257.666667
California,Los Angeles,236294.666667,242379.333333,248779.333333,255336.333333,261622.000000,268508.666667,274942.000000,282457.000000,291412.666667,302555.333333,...,673716.666667,691421.666667,704511.666667,711289.666667,715214.666667,713399.666667,711071.000000,715913.333333,725811.333333,744185.000000
Texas,Houston,106242.666667,106365.666667,107054.000000,108204.666667,108867.333333,108614.000000,108833.000000,109410.333333,110188.000000,111795.333333,...,173392.333333,175054.333333,177529.666667,180100.333333,182791.000000,185761.000000,186502.333333,188010.000000,189638.666667,191080.000000
Illinois,Chicago,157851.000000,162824.333333,167271.666667,171826.333333,176428.333333,181054.666667,185542.000000,190079.666667,194006.666667,197951.000000,...,240331.333333,243683.333333,244584.000000,246298.333333,248184.000000,247895.666667,248254.333333,248618.000000,248132.333333,248657.000000
Texas,San Antonio,102291.000000,101794.333333,100507.666667,100411.666667,100022.000000,99548.333333,99589.666667,99274.666667,99337.333333,100250.000000,...,168950.333333,169737.000000,172521.000000,175068.000000,176882.666667,180058.666667,182096.666667,184042.000000,186051.333333,187269.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Minnesota,Winton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,78597.666667,79370.000000,75008.333333,74156.666667,72650.666667,70245.000000,70529.666667,69230.666667,69284.000000,69802.666667
Mississippi,Eastabuchie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61574.666667,61921.333333,65128.666667,69135.000000,73386.333333,76450.333333,77860.000000,79176.333333,79931.000000,81708.000000
Texas,Dean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,164850.666667,166380.333333,169821.000000,171560.333333,173843.666667,175357.666667,177847.333333,180606.000000,182046.000000,182588.000000


In [9]:
def start_of_recession():
    
    """This function takes GDP and the time as input and 
    returns the year and quarter for the start of the recession"""
    
    gdp = pd.read_excel('gdplev.xlsx',header = 219,names = ['Time', 'GDP','GDP_Chained'])
    req_columns = ['Time','GDP_Chained']
    gdp = gdp[req_columns].set_index(gdp['Time'])
    gdp = gdp['GDP_Chained']
    rec = None
    
    for i in range(0,len(gdp)-2):
        if (gdp[i] > gdp[i+1]) and (gdp[i+1] > gdp[i+2]):
            rec = i
            break
    return gdp.index[rec]

start_of_recession()

'2008Q2'

In [10]:
def end_of_recession():
    
    """This function takes GDP and the time as input and 
    returns the year and quarter for the end of the recession"""

    gdp = pd.read_excel('gdplev.xlsx',header = 219,names = ['Time', 'GDP','GDP_Chained'])
    req_columns = ['Time','GDP_Chained']
    gdp = gdp[req_columns].set_index(gdp['Time'])
    gdp = gdp['GDP_Chained']
    rec = None
    end = None
    
    for i in range(0,len(gdp)-2):
        if (gdp[i] > gdp[i+1]) & (gdp[i+1] > gdp[i+2]):
            rec = i
            break
    for i in range(rec,len(gdp)-2):
        if (gdp[i+2] > gdp[i+1]) & (gdp[i+1] > gdp[i]):
            end = i+2
            break

    return gdp.index[end]

end_of_recession()

'2009Q4'

In [11]:
def bottom_of_recession():
    
    """This function takes GDP and the time as input and 
    returns the year and quarter for the bottom of the recession"""

    gdp = pd.read_excel('gdplev.xlsx',header = 219,names = ['Time', 'GDP','GDP_Chained'])
    req_columns = ['Time','GDP_Chained']
    gdp = gdp[req_columns].set_index(gdp['Time'])
    gdp = gdp['GDP_Chained']
    rec = None
    end = None
    
    for i in range(0,len(gdp)-2):
        if (gdp[i] > gdp[i+1]) & (gdp[i+1] > gdp[i+2]):
            rec = i
            break
    for i in range(rec,len(gdp)-2):
        if (gdp[i+2] > gdp[i+1]) & (gdp[i+1] > gdp[i]):
            end = i+2
            break
    mini = str(gdp[gdp == min(gdp[rec:end])].index.values)[2:8]
    
    return mini

bottom_of_recession()

'2009Q2'

In [12]:
def t_test():
    
    """This function calls the all the functions created above.
    This function runs a t-test to compare the ratio of the mean price of houses in university towns 
    the quarter before the recession starts compared to the recession bottom. 
    (price_ratio=quarter_before_recession/recession_bottom"""
    
    begin = start_of_recession()
    ending = end_of_recession()
    low = bottom_of_recession()
    house = house_year_to_quarters()
    
    
    prices_begin = house[begin]
    prices_end = house[ending]
    ratio = prices_begin.divide(prices_end)
        
    ratio_college = ratio[list(s.index)].dropna()
    
    ratio_not_college_indices = set(house.index) - set(ratio_college.index)
    
    ratio_not_college = ratio.loc[list(ratio_not_college_indices)].dropna()
        
    statistic, p_value = tuple(ttest_ind(ratio_college, ratio_not_college))
    
    outcome = statistic < 0
    
    different = p_value < 0.01
    
    better = ["Non University Town", "University Town"]   
    
    return (different, p_value, better[outcome])

In [13]:
a,b,c=t_test()
print(a,"\nP-Value:",b,"\nThe better performing town is: ",c)

True 
P-Value: 2.8308851059165725e-122 
The better performing town is:  Non University Town
